In [1]:
'''
challenge: resnet input: 3 channels images, the mri images: 20 channels
-> 1 layer (conv2d) to transform 16 to 3 berfore resnet model

challenge: using pretrained resent50 model and update all parameters (aksh-ai)
or just fine tune last layer
-> update all parameters
challenge: resnet-50 transform normalizing is not applicable
we have gray scale 18 channel data 
'''


'\nchallenge: resnet input: 3 channels images, the mri images: 20 channels\n-> 1 layer (conv2d) to transform 16 to 3 berfore resnet model\n\nchallenge: using pretrained resent50 model and update all parameters (aksh-ai)\nor just fine tune last layer\n-> update all parameters\nchallenge: resnet-50 transform normalizing is not applicable\nwe have gray scale 18 channel data \n'

In [2]:
import os

import numpy as np
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader

from config import Config, Device
from datasets import MRIDataset
from models import MriResentModel
from train import Trainer
from test_file import Tester

In [3]:
device = Device.device
print(device)

mps


In [4]:
data_path = os.path.join(os.getcwd(), "data")
labels_path = "train.csv"

batch_size = Config.batch_size
num_epochs = Config.num_epochs
learning_rate = Config.learning_rate
image_size = 256

In [5]:
train_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((image_size, image_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((image_size, image_size)),
    # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [6]:
train_dataset = MRIDataset(
    data_path, labels_path, split="train", transform=train_transforms, max_slices=20)
    
val_dataset = MRIDataset(data_path, labels_path, split="val",
                         transform=test_transforms, max_slices=20)
test_dataset = MRIDataset(
    data_path, labels_path, split="test", transform=test_transforms, max_slices=20)


train_dl = DataLoader(train_dataset, batch_size, shuffle=True)
val_dl = DataLoader(val_dataset, batch_size)
test_dl = DataLoader(test_dataset, batch_size)

In [7]:
model = MriResentModel(20,1).to(device)
model

/Users/fateme/Projects/iaaa-project/iaaa-venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/fateme/Projects/iaaa-project/iaaa-venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MriResentModel(
  (conv): Conv2d(20, 3, kernel_size=(1, 1), stride=(1, 1))
  (resnet_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        

In [8]:
def compute_class_weights_from_csv(csv_file_path):
    # Read the CSV file
    df = pd.read_csv(csv_file_path)

    labels = df['prediction'].values

    # Convert labels to integers if they are not already
    labels = labels.astype(int)

    # Compute class weights
    unique_labels = np.unique(labels)
    class_weights = compute_class_weight(
        class_weight='balanced', classes=unique_labels, y=labels)

    # Convert to torch tensor
    return torch.tensor(class_weights, dtype=torch.float)


# Path to your CSV file
class_weights = compute_class_weights_from_csv(labels_path)
print(class_weights)
# For binary classification, use the appropriate class weight
# Assuming binary classification with class labels 0 and 1
class_weights = class_weights[1]  # Adjust if necessary
print("Class Weights:", class_weights)

tensor([0.5713, 4.0051])
Class Weights: tensor(4.0051)


In [9]:
# loss and optimizer

# criterion = nn.BCEWithLogitsLoss().to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights).to(device)
# criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)

In [10]:
model_name = model.__class__.__name__
model_name

'MriResentModel'

In [11]:
trainer = Trainer(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    train_dl=train_dl,
    val_dl=val_dl,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    device=device,
    num_epochs=100,
    patience=50,
    threshold=0,
    save_path=f"saved_models/{model_name}.pth"
)

# Start training
trainer.train()

100%|██████████| 59/59 [03:03<00:00,  3.10s/it]


Confusion Matrix:
[[  0 548]
 [  0  78]]
Epoch 1/100, Train Loss: 108.5508, Train Accuracy: 0.1250
Epoch 1/100, Val Accuracy: 0.1246, Precision: 0.1246, Recall: 1.0000, AUC: 0.5000, Avg Metric: 0.5415


100%|██████████| 59/59 [02:58<00:00,  3.03s/it]


Confusion Matrix:
[[  0 548]
 [  0  78]]
Epoch 2/100, Train Loss: 125.3991, Train Accuracy: 0.1255
Epoch 2/100, Val Accuracy: 0.1246, Precision: 0.1246, Recall: 1.0000, AUC: 0.5139, Avg Metric: 0.5462


100%|██████████| 59/59 [03:00<00:00,  3.05s/it]


...

Confusion Matrix:
[[  0 548]
 [  0  78]]
Epoch 92/100, Train Loss: 43.5474, Train Accuracy: 0.1250
Epoch 92/100, Val Accuracy: 0.1246, Precision: 0.1246, Recall: 1.0000, AUC: 0.5579, Avg Metric: 0.5609


100%|██████████| 59/59 [02:48<00:00,  2.85s/it]


Confusion Matrix:
[[  0 548]
 [  0  78]]
Epoch 93/100, Train Loss: 41.8880, Train Accuracy: 0.1250
Epoch 93/100, Val Accuracy: 0.1246, Precision: 0.1246, Recall: 1.0000, AUC: 0.5259, Avg Metric: 0.5502


 46%|████▌     | 27/59 [01:20<01:34,  2.97s/it]


KeyboardInterrupt: 

In [12]:
model = MriResentModel(20,1).to(device)
model.load_state_dict(torch.load(f"saved_models/{model_name}.pth"))

/Users/fateme/Projects/iaaa-project/iaaa-venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/fateme/Projects/iaaa-project/iaaa-venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<All keys matched successfully>

In [13]:
tester = Tester(
    model=model,
    test_dl=test_dl,
    test_dataset=test_dataset,
    device=device,
    threshold=0  # Set the threshold for binary classification
)

# Perform testing and print metrics
tester.test()

Test Accuracy: 0.7604, Precision: 0.2049, Recall: 0.3205, AUC: 0.5926, Avg Metric: 0.3727
